In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
image_dir = Path('C:/Users/asus/OneDrive/Desktop/Project/Skin Disease')

In [3]:
image_dir

WindowsPath('C:/Users/asus/OneDrive/Desktop/Project/Skin Disease')

In [4]:
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)

In [5]:
[os.path.abspath(filepaths[0]) for filepaths[0] in filepaths ]


['C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024329.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024372.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024418.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024450.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024463.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024468.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024470.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024511.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024517.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024522.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Project\\Skin Disease\\AKIEC\\ISIC_0024539.jpg',
 'C:\\Users\\asus\\OneDrive\\Desktop\\Proje

In [6]:
images.Label.value_counts()

Label
NV       6457
BKL      1055
MEL      1015
BCC       495
AKIEC     327
VASC      138
DF        113
Name: count, dtype: int64

In [7]:
images

,Filepath,Label
0,C:\Users\asus\OneDrive\Desktop\Project\Skin Di...,AKIEC
1,C:\Users\asus\OneDrive\Desktop\Project\Skin Di...,AKIEC
2,C:\Users\asus\OneDrive\Desktop\Project\Skin Di...,AKIEC
3,C:\Users\asus\OneDrive\Desktop\Project\Skin Di...,AKIEC
4,C:\Users\asus\OneDrive\Desktop\Project\Skin Di...,AKIEC
...,...,...
9595,C:\Users\asus\OneDrive\Desktop\Project\Skin Di...,VASC
9596,C:\Users\asus\OneDrive\Desktop\Project\Skin Di...,VASC
9597,C:\Users\asus\OneDrive\Desktop\Project\Skin Di...,VASC
9598,C:\Users\asus\OneDrive\Desktop\Project\Skin Di...,VASC


In [8]:
train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)

In [9]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.2)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function = tf.keras.applications.resnet50.preprocess_input)

In [10]:
train_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (244,244),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col ='Filepath',
    y_col ='Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = False
)

Found 5376 validated image filenames belonging to 7 classes.
Found 1344 validated image filenames belonging to 7 classes.
Found 2880 validated image filenames belonging to 7 classes.


In [11]:
pretrained_model = tf.keras.applications.resnet50.ResNet50(
    input_shape = (244,244,3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'avg'
)

pretrained_model.trainable = False

In [12]:
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(50, activation='relu')(x)
outputs = tf.keras.layers.Dense(7, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 244, 244, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 250, 250, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 122, 122, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 122, 122, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 61, 61, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 61, 61, 64)           256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, 61, 61, 64)           0         ['conv2_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_blo

 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 31, 31, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, 31, 31, 512)          2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_add (Add)      (None, 31, 31, 512)          0         ['conv3_block1_out[0][0]',    
                                                                     'conv3_block2_3_bn[0][0]']   
          

 conv4_block1_2_conv (Conv2  (None, 16, 16, 256)          590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, 16, 16, 256)          1024      ['conv4_block1_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_2_relu (Activ  (None, 16, 16, 256)          0         ['conv4_block1_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_0_conv (Conv2  (None, 16, 16, 1024)         525312    ['conv3_block4_out[0][0]']    
 D)       

                                                                                                  
 conv4_block3_out (Activati  (None, 16, 16, 1024)         0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block4_1_conv (Conv2  (None, 16, 16, 256)          262400    ['conv4_block3_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block4_1_bn (BatchNo  (None, 16, 16, 256)          1024      ['conv4_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_blo

 D)                                                                                               
                                                                                                  
 conv4_block6_3_bn (BatchNo  (None, 16, 16, 1024)         4096      ['conv4_block6_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block6_add (Add)      (None, 16, 16, 1024)         0         ['conv4_block5_out[0][0]',    
                                                                     'conv4_block6_3_bn[0][0]']   
                                                                                                  
 conv4_block6_out (Activati  (None, 16, 16, 1024)         0         ['conv4_block6_add[0][0]']    
 on)                                                                                              
          

 conv5_block3_1_relu (Activ  (None, 8, 8, 512)            0         ['conv5_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block3_2_conv (Conv2  (None, 8, 8, 512)            2359808   ['conv5_block3_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv5_block3_2_bn (BatchNo  (None, 8, 8, 512)            2048      ['conv5_block3_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block3_2_relu (Activ  (None, 8, 8, 512)            0         ['conv5_block3_2_bn[0][0]']   
 ation)   

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(train_images, validation_data = val_images, epochs=25, callbacks=[callbacks])

Epoch 1/25


168/168 [==============================] - 938s 6s/step - loss: 0.7931 - accuracy: 0.7184 - val_loss: 0.7509 - val_accuracy: 0.7381
Epoch 2/25
168/168 [==============================] - 996s 6s/step - loss: 0.5894 - accuracy: 0.7805 - val_loss: 0.7503 - val_accuracy: 0.7321
Epoch 3/25
168/168 [==============================] - 984s 6s/step - loss: 0.5115 - accuracy: 0.8123 - val_loss: 0.7209 - val_accuracy: 0.7433
Epoch 4/25
168/168 [==============================] - 1015s 6s/step - loss: 0.4597 - accuracy: 0.8298 - val_loss: 0.6119 - val_accuracy: 0.7842
Epoch 5/25
168/168 [==============================] - 988s 6s/step - loss: 0.4080 - accuracy: 0.8462 - val_loss: 0.5984 - val_accuracy: 0.7924
Epoch 6/25
168/168 [==============================] - 997s 6s/step - loss: 0.3598 - accuracy: 0.8690 - val_loss: 0.6268 - val_accuracy: 0.7939
Epoch 7/25
168/168 [==============================] - 909s 5s/step - loss: 0.3386 - accuracy: 0.8707 - val_loss: 0.6098 - val_accuracy: 0.7

In [15]:
results = model.evaluate(test_images, verbose = 0)
print(results)
print(f"test accuracy: {np.round(results[1] * 100, 2)}%")

[0.6413666605949402, 0.7711805701255798]
test accuracy: 77.12%


In [16]:
predictions = np.argmax(model.predict(test_images), axis=1)
matrix = confusion_matrix(test_images.labels, predictions)
report= classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

90/90 [==============================] - 349s 4s/step


In [18]:
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

       AKIEC       0.46      0.57      0.51        98
         BCC       0.67      0.43      0.52       142
         BKL       0.60      0.57      0.58       297
          DF       0.75      0.15      0.24        41
         MEL       0.44      0.54      0.48       324
          NV       0.88      0.89      0.89      1938
        VASC       0.96      0.55      0.70        40

    accuracy                           0.77      2880
   macro avg       0.68      0.53      0.56      2880
weighted avg       0.78      0.77      0.77      2880

